# ROB 204 - Communication Lab

In [1]:
import os, sys, math, threading, time, logging
import cv2
from picamera2 import Picamera2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import ipywidgets as widgets
from IPython import display
from gpiozero import AngularServo as Servo
from gpiozero.pins.pigpio import PiGPIOFactory

# Helper Functions
pigpio_factory = PiGPIOFactory()
EmbodimentYawServo = Servo(13, pin_factory=pigpio_factory, min_angle = 80, max_angle = -100, min_pulse_width=0.0005, max_pulse_width=0.0024)

def setEmbodimentYaw(degrees):
    global EmbodimentYawServo
    EmbodimentYawServo.angle = (degrees)

setEmbodimentYaw(0)
    
# FUNCTION GENERATED WITH CHATGPT -- REVISE!!!
def rotation_matrix_to_euler_angles(R):
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.array([x, y, z])
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

def process_rotation(yaw):
    global tracking_output
    if yaw > 20:
        tracking_output.value = "👈"
        setEmbodimentYaw(-45)
    elif yaw < -20:
        tracking_output.value = "👉"
        setEmbodimentYaw(45)
    else:
        tracking_output.value = "🫵"
        setEmbodimentYaw(0)

os.environ["LIBCAMERA_LOG_LEVELS"] = "3"

base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=False,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

tracking = None
tracking_run = False

tracking_status = widgets.HTML(value="Tracking not started")

file = open("images/placeholder.jpg", "rb")
image = file.read()
tracking_image  = widgets.Image(value=image,format='jpeg',width=640,height=480)

tracking_output = widgets.HTML(value="🙈")
tracking_fps = widgets.HTML(value="FPS: 0")

disp_container = widgets.VBox([tracking_status, tracking_image, tracking_output, tracking_fps])
output = widgets.Output()

TARGET_FPS = 10

def tracking_function(tracking_status, tracking_image, tracking_output, tracking_fps):
    tracking_status.value = "Tracking starting..."
    with Picamera2() as picam:
        picam.configure(picam.create_video_configuration(main={"format": 'RGB888', "size": (1296, 730)}))
        picam.start()
        tracking_status.value = "Tracking started."
        while tracking_run:
            start_time = time.time()
            image = picam.capture_array("main")
    
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
            detection_result = detector.detect(mp_image)

            resize_image = cv2.flip(image, 1)
            resize_frame = cv2.resize(resize_image, (0, 0), fx = 0.25, fy = 0.25)
            _,ret_array = cv2.imencode('.jpg', resize_frame)
            tracking_image.value = ret_array
            
            try:
                transformation_matrix = detection_result.facial_transformation_matrixes[0]
                rotation_angles = rotation_matrix_to_euler_angles(transformation_matrix)
                yaw_angle = np.degrees(rotation_angles[1])     
                process_rotation(yaw_angle)
    
            except Exception as e:
                tracking_output.value = "🙈"
                pass

            fps = 1.0 / (time.time() - start_time)
            tracking_fps.value = "FPS: " + str(fps)
            sleep_time = (1.0 / TARGET_FPS) - (time.time() - start_time)
            if sleep_time > 0:
                time.sleep(sleep_time)

            adj_fps = 1.0 / (time.time() - start_time)
            tracking_fps.value = "FPS: " + str(adj_fps)

def start_tracking(b):
    global tracking, tracking_run, disp_container
    with output:
        if tracking is None or not tracking.is_alive():
            tracking_run = True
            tracking = threading.Thread(target=tracking_function, args=(tracking_status, tracking_image, tracking_output, tracking_fps, ))
            tracking.start()

def stop_tracking(b):
    global tracking, tracking_run
    with output:
        if tracking is not None and tracking.is_alive():
            tracking_run = False
            tracking_status.value = "Tracking stopped."

display.clear_output()
start_btn = widgets.Button(description="Start Tracking", button_style='success')
start_btn.on_click(start_tracking)

stop_btn = widgets.Button(description="Stop Tracking", button_style='danger')
stop_btn.on_click(stop_tracking)

toolbar = widgets.HBox([start_btn, stop_btn])
disp = widgets.Box(children = [widgets.VBox([toolbar, disp_container])], layout=widgets.Layout(justify_content='center'))
display.display(disp, output)

Box(children=(VBox(children=(HBox(children=(Button(button_style='success', description='Start Tracking', style…

Output()

## Editable Function

In [2]:
def process_rotation(yaw):
    global tracking_output
    if yaw > 20:
        tracking_output.value = "👈"
        setEmbodimentYaw(-45)
    elif yaw < -20:
        tracking_output.value = "👉"
        setEmbodimentYaw(45)
    else:
        tracking_output.value = "🫵"
        setEmbodimentYaw(0)